# Multi-variable linear regression

In [ ]:
import opengrid as og
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
df = og.datasets.get('gas_2016_hour')
dfw = og.datasets.get('weather_2016_hour')

### We want to create a model for one of the gas counters. We will create both a daily and a monthly model. 

First we have to compute (different definitions of) degree-days. We also put together both dataframes to get a single dataframe. 

In [ ]:
dfw = dfw.resample('D').mean()
dfw_HDD = og.library.weather.compute_degree_days(ts=dfw['temperature'],
                                                  heating_base_temperatures=range(8, 18, 2),
                                                  cooling_base_temperatures=range(16, 26, 2)).bfill()
dfw_HDD = pd.concat([dfw, dfw_HDD], axis=1)

df_day = df.resample('D').sum()/1000. # kWh/day
df_day = pd.concat([df_day, dfw_HDD], axis=1).loc['2016']
df_month = df_day.resample('MS').sum()
print(df_day.info())
print(df_month.info())

In [ ]:
list_of_exog = df_month.columns.tolist()
# remove columns we don't want to include in the regression analysis
fo
mvlr = og.MultiVarLinReg(df_month, endog='313b', list_of_exog=list_of_exog)

In [ ]:
print(mvlr.fit.summary())
mvlr.plot()